# (EX) News article processing (with ML pipeline)

In [ ]:
# spark setup, only if you need to specify your paths

import os
os.environ["JAVA_HOME"] = r"C:\Program Files\Java\jdk-11.0.2"
os.environ["SPARK_HOME"] = r"C:\Program Files\Spark\spark-3.5.5-bin-hadoop3"

In [ ]:
# findspark helps locate the environment variables
import findspark
findspark.init()

# `agnews` Dataset

In [ ]:
!curl https://raw.githubusercontent.com/mosesyhc/de300-2025sp-class/refs/heads/main/agnews.csv -O

# Pipelining with PySpark MLlib

In [ ]:
from pyspark.sql import SparkSession
from pyspark.ml import Pipeline # pipeline to transform data


In [ ]:
spark = (SparkSession.builder
         .master("local[*]")
         .appName("AG news")
         .getOrCreate()
        )
sc = spark.sparkContext

In [ ]:
# load dataset
df = spark.read.csv("agnews.csv", inferSchema=True, header=True)

In [ ]:
df.show(20)

# Arrange columns

In [ ]:
from pyspark.sql.functions import concat_ws, col # to concatinate cols

# renaming 'Class Index' col to 'label'
df = df.withColumnRenamed('Class Index', 'label')

# concatenating texts
df = df.withColumn('text', concat_ws(" ", "Title", "Description"))

df = df.select('label', 'text')

df.show(10)

# Tokenize

In [ ]:
from pyspark.ml.feature import RegexTokenizer # tokenizer

# convert sentences to list of words
tokenizer = RegexTokenizer(inputCol='text', outputCol='words', pattern="\\W")

df = tokenizer.transform(df)

df.select(['label', 'text', 'words']).show(5)

# Stopwords

In [ ]:
from pyspark.ml.feature import StopWordsRemover

# remove stopwords
stopwords_remover = StopWordsRemover(inputCol="words", outputCol="filtered")

df = stopwords_remover.transform(df)


df.select(['label', 'text', 'words', 'filtered']).show(5)

# Term frequency, Inverse document frequency

In [ ]:
from pyspark.ml.feature import HashingTF

# calculate term frequency in each article (row)

hashing_tf = HashingTF(inputCol="filtered", outputCol="raw_features",
                       numFeatures=16384)

featurized_data = hashing_tf.transform(df)

featurized_data.select('raw_features').show(5)

In [ ]:
from pyspark.ml.feature import IDF

# inverse document frequency
idf = IDF(inputCol="raw_features", outputCol="features")

idf_vectorizer = idf.fit(featurized_data)

rescaled_data = idf_vectorizer.transform(featurized_data)

rescaled_data.show(5)

In [ ]:
rescaled_data.select('raw_features').show(2, truncate=False)
rescaled_data.select('features').show(2, truncate=False)

# Training a multinomial logistic regression

In [ ]:
# split data
(train, test) = rescaled_data.randomSplit([0.75, 0.25], seed=42)
train.count()

In [ ]:
rescaled_data.printSchema()

In [ ]:
from pyspark.ml.classification import LogisticRegression

lr = LogisticRegression(featuresCol='features',
                        labelCol='label',
                        family='multinomial',
                        regParam=0.3,
                        elasticNetParam=0,
                        maxIter=20)

lrModel = lr.fit(train)

# Prediction and evaluation

In [ ]:
# predict on test data
predictions = lrModel.transform(test)

In [ ]:
predictions.select('probability', 'prediction').show(5, truncate=False)

In [ ]:
predictions = predictions.withColumn('correctFlag', (col('label') == col('prediction')))

In [ ]:
from pyspark.sql.functions import avg
from pyspark.sql.types import FloatType

predictions.select(avg(col('correctFlag').cast(FloatType())).alias('acurracy')).show()

In [ ]:
from pyspark.mllib.evaluation import MulticlassMetrics
# labels = ["World", "Sports", "Business","Science"]

# take only the predictions
preds_and_labels = predictions.select(['prediction','label']) \
                              .withColumn('label', col('label') \
                              .cast(FloatType())) \
                              .orderBy('prediction')


preds_and_labels.show(5)

In [ ]:
# confusion matrix
metrics = MulticlassMetrics(predictionAndLabels=preds_and_labels.rdd.map(tuple))

In [ ]:
metrics.confusionMatrix().toArray()

# Pipelining, from start to finish

In [ ]:
# load dataset
df = spark.read.csv("agnews.csv", inferSchema=True, header=True)

def arrangeColumns(df):
  # Renaming 'Class Index' col to 'label'
  df = df.withColumnRenamed('Class Index', 'label')

  # Add a new column 'text' by joining 'Title' and 'Description'
  df = df.withColumn("text", concat_ws(" ", "Title", 'Description'))

  # Select new text feature and labels
  df = df.select('label', 'text')
  return df

df = arrangeColumns(df)

# tokenizer
tokenizer = RegexTokenizer(inputCol="text", outputCol="words", pattern="\\W")

# stopwords
stopwords_remover = StopWordsRemover(inputCol="words", outputCol="filtered")

# term frequency
hashing_tf = HashingTF(inputCol="filtered",
                       outputCol="raw_features",
                       numFeatures=16384)

# Inverse Document Frequency
idf = IDF(inputCol="raw_features", outputCol="features")

# model
lr = LogisticRegression(featuresCol='features',
                        labelCol='label',
                        family="multinomial",
                        regParam=0.3,
                        elasticNetParam=0,
                        maxIter=20)



In [ ]:
# Put everything in pipeline
pipeline = Pipeline(stages=[tokenizer,
                            stopwords_remover,
                            hashing_tf,
                            idf,
                            lr])

# Fit the pipeline to training documents.
pipelineFit = pipeline.fit(df)

# transform and train
dataset = pipelineFit.transform(df)